In [ ]:
!pip install tensorflow

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

In [5]:
dataset_dir = "D:/Academic/4th year/Research/NewDataset"  # Update with the correct path
train_dir = os.path.join(dataset_dir, 'train')
test_dir = os.path.join(dataset_dir, 'test')

In [7]:
# Image data generators for augmentation and preprocessing
img_height, img_width = 150, 150
batch_size = 32

In [11]:
# Using validation_split to split training data into training and validation sets
train_datagen = ImageDataGenerator(rescale=1./255, 
                                   rotation_range=20, 
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   validation_split=0.2)  # 20% for validation
# Training data generator
train_generator = train_datagen.flow_from_directory(train_dir,
                                                     target_size=(img_height, img_width),
                                                     batch_size=batch_size,
                                                     class_mode='categorical',
                                                     subset='training')  # Specify training subset
# Validation data generator
val_generator = train_datagen.flow_from_directory(train_dir,
                                                   target_size=(img_height, img_width),
                                                   batch_size=batch_size,
                                                   class_mode='categorical',
                                                   subset='validation')  # Specify validation subset
# Test data generator
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(test_dir,
                                                   target_size=(img_height, img_width),
                                                   batch_size=batch_size,
                                                   class_mode='categorical',
                                                   shuffle=False)

Found 6954 images belonging to 13 classes.
Found 1733 images belonging to 13 classes.
Found 2170 images belonging to 13 classes.


In [13]:
# Step 3: Define CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(13, activation='softmax')  # 13 classes
])

C:\Users\HP\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [15]:
# Step 4: Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [17]:
# Step 5: Train the model
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model.fit(train_generator,
                    validation_data=val_generator,
                    epochs=30,
                    callbacks=[early_stopping]
                   )

C:\Users\HP\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
218/218 ━━━━━━━━━━━━━━━━━━━━ 0s 7s/step - accuracy: 0.3517 - loss: 1.8873

C:\Users\HP\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


218/218 ━━━━━━━━━━━━━━━━━━━━ 2174s 10s/step - accuracy: 0.3524 - loss: 1.8854 - val_accuracy: 0.6549 - val_loss: 1.1121
Epoch 2/30
218/218 ━━━━━━━━━━━━━━━━━━━━ 1912s 9s/step - accuracy: 0.7119 - loss: 0.8508 - val_accuracy: 0.7571 - val_loss: 0.9191
Epoch 3/30
218/218 ━━━━━━━━━━━━━━━━━━━━ 1861s 9s/step - accuracy: 0.7887 - loss: 0.6076 - val_accuracy: 0.6844 - val_loss: 1.1394
Epoch 4/30
218/218 ━━━━━━━━━━━━━━━━━━━━ 1530s 7s/step - accuracy: 0.8369 - loss: 0.4632 - val_accuracy: 0.7634 - val_loss: 0.8798
Epoch 5/30
218/218 ━━━━━━━━━━━━━━━━━━━━ 1534s 7s/step - accuracy: 0.8658 - loss: 0.4147 - val_accuracy: 0.7744 - val_loss: 0.9297
Epoch 6/30
218/218 ━━━━━━━━━━━━━━━━━━━━ 1536s 7s/step - accuracy: 0.8896 - loss: 0.3411 - val_accuracy: 0.7877 - val_loss: 0.8994
Epoch 7/30
218/218 ━━━━━━━━━━━━━━━━━━━━ 1519s 7s/step - accuracy: 0.8964 - loss: 0.3227 - val_accuracy: 0.7640 - val_loss: 1.2053
Epoch 8/30
218/218 ━━━━━━━━━━━━━━━━━━━━ 2322s 11s/step - accuracy: 0.9151 - loss: 0.2626 - val_accur

In [19]:
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Accuracy: {test_accuracy:.2f}")

68/68 ━━━━━━━━━━━━━━━━━━━━ 448s 7s/step - accuracy: 0.6844 - loss: 2.0997
Test Accuracy: 0.67


In [ ]:
model_save_path = "leaf_identifyer.h5"
model.save(model_save_path)
print(f"Model saved at {model_save_path}")

In [ ]:
# Step 8: Plot accuracy
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
from tensorflow.keras.models import load_model
def load_and_test_model(image_path):
    loaded_model = load_model(model_save_path)
    from tensorflow.keras.preprocessing import image
    img = image.load_img(image_path, target_size=(img_height, img_width))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    prediction = loaded_model.predict(img_array)
    class_indices = {v: k for k, v in train_generator.class_indices.items()}
    predicted_class = class_indices[np.argmax(prediction)]
    print(f"Predicted leaf type: {predicted_class}")


In [ ]:
image_path = 'g2.jpg'
load_and_test_model(image_path)

In [23]:
import os
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from PIL import Image, ImageDraw, ImageFont

# Constants
model_save_path = "leaf_identifyer.h5"
img_height, img_width = 224, 224  # Ensure these match your model's input size
output_folder = "D:/Academic/4th year/Research/results"  # Folder to save annotated images

# Load the trained model
def load_and_save_predictions(folder_path):
    # Load the saved model
    loaded_model = load_model(model_save_path)
    
    # Reverse the class indices to map predictions to class names
    class_indices = {v: k for k, v in train_generator.class_indices.items()}
    
    # Create output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # Iterate over all files in the folder
    for img_file in os.listdir(folder_path):
        img_path = os.path.join(folder_path, img_file)
        
        if not img_file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tif', '.tiff')):
            # Skip files that are not images
            print(f"Skipping non-image file: {img_file}")
            continue
        
        # Load and preprocess the image
        img = image.load_img(img_path, target_size=(img_height, img_width))
        img_array = image.img_to_array(img) / 255.0
        img_array = np.expand_dims(img_array, axis=0)
        
        # Predict the class
        prediction = loaded_model.predict(img_array)
        predicted_class = class_indices[np.argmax(prediction)]
        
        # Annotate and save the image
        save_annotated_image(img_path, img_file, predicted_class)

# Function to annotate and save the image
def save_annotated_image(img_path, img_file, predicted_class):
    # Open the original image
    original_img = Image.open(img_path)
    
    # Create a drawing context
    draw = ImageDraw.Draw(original_img)
    
    # Define font (use default if custom font isn't available)
    try:
        font = ImageFont.truetype("arial.ttf", size=20)
    except IOError:
        font = ImageFont.load_default()
    
    # Add the prediction text
    text = f"Prediction: {predicted_class}"
    text_position = (10, 10)  # Top-left corner
    text_color = (255, 0, 0)  # Red color
    
    draw.text(text_position, text, fill=text_color, font=font)
    
    # Save the annotated image in the output folder
    output_path = os.path.join(output_folder, img_file)
    original_img.save(output_path)
    print(f"Saved annotated image: {output_path}")

# Folder path containing test images
folder_path = 'D:/Academic/4th year/Research/test'
load_and_save_predictions(folder_path)


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Saved annotated image: D:/Academic/4th year/Research/results\cinnamon_leaf_1.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
Saved annotated image: D:/Academic/4th year/Research/results\cinnamon_leaf_10.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
Saved annotated image: D:/Academic/4th year/Research/results\cinnamon_leaf_2.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
Saved annotated image: D:/Academic/4th year/Research/results\cinnamon_leaf_3.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
Saved annotated image: D:/Academic/4th year/Research/results\cinnamon_leaf_4.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
Saved annotated image: D:/Academic/4th year/Research/results\cinnamon_leaf_5.jpeg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
Saved annotated image: D:/Academic/4th year/Research/results\cinnamon_leaf_6.jpeg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
Saved annotated image: D:/Academic/4th year/Research/results\cinnamon_leaf_7.jpeg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
Saved an